In [1]:
from IPython.core.pylabtools import figsize

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import h5py, pylab
from tqdm import tqdm

figsize(16,9)

In [2]:
import os
import ipyparallel as ipp

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

TimeoutError: Hub connection request timed out

In [4]:
import os
import ipyparallel as ipp

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

ipython --matplotlib
%matplotlib

TimeoutError: Hub connection request timed out

In [5]:
def mkmovie(images, outimg=None, fps=5, size=None,is_color=True, format="XVID"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

def cineplt(data):
    plt.figure(figsize = (12,12))
    img = data
    imgplot = plt.imshow(img)
    plt.colorbar()
    return img

def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [6]:
with h5py.File('test.h5','r') as hf:
    hf_data = hf.get('images')
    raw = np.array(hf_data)
    
with h5py.File('dcoff.h5','r') as hf:
    hf_data = hf.get('images')
    fluc = np.array(hf_data)

In [8]:
%%capture
rawimgs = []
K = len(raw[1,1,:])
K=100

for k in tqdm(range(0,K)):
    cineframe = cineplt(raw[:,:,k])
    pylab.savefig('CineFrames/cineframe%03d.png'%k)
    rawimgs.append(cineframe)

100%|██████████| 100/100 [00:31<00:00,  3.28it/s]


In [20]:
from ffmpy import FFmpeg
ff = FFmpeg(
    inputs={'image2': ['-f']},
    outputs={'movie.mp4': ['-r','1/5','-i','cineframe%03d.png','-vcodec','mpeg4','-y']})
print ff.cmd

ffmpeg -f -i image2 -r 1/5 -i cineframe%03d.png -vcodec mpeg4 -y movie.mp4


In [27]:
import cv2

ImportError: DLL load failed: %1 is not a valid Win32 application.

In [ ]:
for i in range(len(rawimgs)):
    rawimgs[i].save('CineFrames/cineframe%03d.png'%i)

In [ ]:
import moviepy.editor as mpy
clip = mpy.ImageSequenceClip(rawimgs, fps=24)
clip.write_videofile('movie.mp4')

In [ ]:
import matplotlib.animation as animation
import numpy as np
from pylab import *


dpi = 100

def ani_frame():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    im = ax.imshow(rand(300,300),cmap='gray',interpolation='nearest')
    im.set_clim([0,1])
    fig.set_size_inches([5,5])


    tight_layout()


    def update_img(n):
        tmp = rand(300,300)
        im.set_data(tmp)
        return im

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,300,interval=30)
    writer = animation.writers['ffmpeg'](fps=30)

    ani.save('demo.mp4',writer=writer,dpi=dpi)
    return ani

In [ ]:
mkmovie(rawimgs, outimg=None, fps=5, size=None,is_color=True, format="XVID")

In [ ]:
def CreateMovie(plotter, numberOfFrames, fps=10):
	import os, sys

	import matplotlib.pyplot as plt
 
	for i in range(numberOfFrames):
		plotter(i)

		fname = '_tmp%05d.png'%i
 
		plt.savefig(fname)
		plt.clf()

	os.system("movie.mp4")

	os.system("ffmpeg -r "+str(fps)+" -b 1800 -i _tmp%05d.png movie.mp4")
	os.system("rm _tmp*.png")

def mkframe(k):
    return cineplt(raw[:,:,k])

In [ ]:
rawimgs(0).shape

In [ ]:
from ipyparallel import Client
cluster = Client(profile="nbserver")

In [ ]:
Microsoft Windows [Version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation.  All rights reserved.

C:\Users\Narwhal>ipcluster start
Traceback (most recent call last):
  File "c:\users\narwhal\anaconda2\lib\runpy.py", line 174, in _run_module_as_ma
in
    "__main__", fname, loader, pkg_name)
  File "c:\users\narwhal\anaconda2\lib\runpy.py", line 72, in _run_code
    exec code in run_globals
  File "C:\Users\Narwhal\Anaconda2\Scripts\ipcluster.exe\__main__.py", line 5, i
n <module>
  File "c:\users\narwhal\anaconda2\lib\site-packages\ipyparallel\apps\ipclustera
pp.py", line 573, in <module>
    class IPClusterNBExtension(BaseIPythonApplication):
  File "c:\users\narwhal\anaconda2\lib\site-packages\ipyparallel\apps\ipclustera
pp.py", line 592, in IPClusterNBExtension
    user = Bool(False, help="Apply the operation only for the given user").tag(
AttributeError: 'Bool' object has no attribute 'tag'

C:\Users\Narwhal>conda install ipyparallel
Fetching package metadata .........
Solving package specifications: ..........

Package plan for installation in environment C:\Users\Narwhal\Anaconda2:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    futures-3.0.5              |           py27_0          18 KB
    pathlib2-2.1.0             |           py27_0          26 KB
    pickleshare-0.7.4          |           py27_0           8 KB
    ipyparallel-5.2.0          |           py27_0         321 KB
    ------------------------------------------------------------
                                           Total:         374 KB

The following NEW packages will be INSTALLED:

    futures:     3.0.5-py27_0
    ipyparallel: 5.2.0-py27_0
    pathlib2:    2.1.0-py27_0

The following packages will be UPDATED:

    pickleshare: 0.5-py27_0   <unknown> --> 0.7.4-py27_0

Proceed ([y]/n)? y

Fetching packages ...
futures-3.0.5- 100% |###############################| Time: 0:00:00 229.26 kB/s
pathlib2-2.1.0 100% |###############################| Time: 0:00:00 326.18 kB/s
pickleshare-0. 100% |###############################| Time: 0:00:00   8.51 MB/s
ipyparallel-5. 100% |###############################| Time: 0:00:00 632.33 kB/s
Extracting packages ...
[      COMPLETE      ]|##################################################| 100%
Unlinking packages ...
[      COMPLETE      ]|##################################################| 100%
Linking packages ...
[      COMPLETE      ]|##################################################| 100%

C:\Users\Narwhal>ipcluster start
Traceback (most recent call last):
  File "C:\Users\Narwhal\Anaconda2\Scripts\ipcluster-script.py", line 3, in <mod
ule>
    import ipyparallel.apps.ipclusterapp
  File "C:\Users\Narwhal\Anaconda2\lib\site-packages\ipyparallel\apps\ipclustera
pp.py", line 573, in <module>
    class IPClusterNBExtension(BaseIPythonApplication):
  File "C:\Users\Narwhal\Anaconda2\lib\site-packages\ipyparallel\apps\ipclustera
pp.py", line 592, in IPClusterNBExtension
    user = Bool(False, help="Apply the operation only for the given user").tag(
AttributeError: 'Bool' object has no attribute 'tag'

C:\Users\Narwhal>

In [2]:
import ipyparallel as ipp
rc = ipp.Client()
cluster = ipp.Client(profile='narwhal')
print('profile:', cluster.profile)
print("IDs:", cluster.ids) # Print process id numbers
dv = rc[:]
dv

IOError: You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [5]:
import os
import ipyparallel as ipp

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

In [6]:
rc
